# Méthodologie

Les résultats du championnat du monde de Backyard Ultra 2023 sont disponibles sur le site https://my.raceresult.com/266852/results

Lorsque l'on clique sur la section "Last finishers", une requête est envoyée à [cette page](https://my1.raceresult.com/266852/RRPublish/data/list?key=c32693b41c83792c4af8a8bcf61c4aa0&listname=Presenter/Announcer|Last%20Finishers&page=results&contest=0&r=all&l=0), qui renvoie le classement de la course, avec le nombre de tours parcourus et les informations de chaque participant, en format JSON, ce qui permet de le traiter rapidement. Ces données sont incorporées dans un dataframe.

Dans ce dataframe, on y ajoute ensuite les données de temps au tour disponibles sur [cette page](https://my1.raceresult.com/266852/RRPublish/data/list?key=c32693b41c83792c4af8a8bcf61c4aa0&listname=Result%20Lists|Lap%20Details&page=results&contest=0&r=all&l=0), qui renvoie de nouveau les données sous format JSON. 

Les temps au tour sont ensuite convertis en secondes et on vérifie que les colonnes correspondantes dans le dataframe ont bien pour dtype des entiers.



# Données supplémentaires

D'après le site contenant les résultats, la course se déroule sur deux circuits différents, l'un durant le jour et l'autre durant la nuit: "Trail Loop 11 Hours, then Road Loop 13 Hours - Repeat as Necessary". On retrouve le profil de ces deux circuits [ici](https://www.strava.com/segments/17318067) et [ici](https://www.strava.com/segments/17318039). 

Nous savons que le tour de jour est en forêt et comporte 154m de dénivelé positif selon Strava. Le tour de nuit est sur route et en comporte 40. Ces deux tours ont pour longueur 4.16667 miles, conformément aux règles du Backyard Ultra. Dans la suite, nous n'utiliserons pas les chiffres exacts de dénivelé positif mais nous gardons à l'esprit que le tour de jour est plus exigeant.

La course a eu lieu du 21 octobre au 25 octobre dans le Tennessee, dans le petit village de Bell Buckle.

Les conditions météorologiques étaient plutôt bonnes, avec un temps sec, des températures autour de 25°C le jour et 10°C la nuit et le vent était relativement faible (cf. [cette page](https://www.wunderground.com/history/weekly/us/tn/nashville/KBNA/date/2023-10-23)). Cependant, ces données ont été enregistrées à l'aéoroport de Nashville, à une cinquantaine de kilométres de la course donc ne reflètent pas forcément la réalité mais donne une idée des conditions.

Comme les conditions météorologiques n'étaient pas particulièrement éprouvantes et que les coureurs étaient abrités du vent et du soleil durant la boucle de jour, nous décidons de ne pas récupérer les données météo dans ce script et de ne pas les incorporer dans l'analyse des données.


# Collection des données

In [30]:
import requests
import pandas as pd
import re

In [14]:
url="https://my1.raceresult.com/266852/RRPublish/data/list?key=c32693b41c83792c4af8a8bcf61c4aa0&listname=Presenter/Announcer|Last Finishers&page=results&contest=0&r=all&l=0"
#call the API that returns JSON. 
page=requests.get(url)
result_json=page.json() #Overview of the results with info on the runners

In [15]:
json.dumps(result_json, indent=4); #to visualize, remove semicolon

In [16]:
col_labels=[a["Label"] for a in result_json["list"]["Fields"]] #getting the labels of columns

In [18]:
df=pd.DataFrame(columns=col_labels)
for i in range(len(result_json["data"])):
   df.loc[len(df)] = result_json["data"][i][1:-1]
df.index+=1 #make sure the index represents the overall position

In [19]:
df

,Place,,Bib,Name,Gender,Age,City,State,Laps,Last Loop,Miles,RaceTime
1,1.,[img:flags/US.gif],5,Harvey Lewis,M,47,Cincinnati,OH,108,47:31,450.00,94:16:36
2,DNC,[img:flags/CA.gif],24,Ihor Verys,M,29,Chilliwack,BC,107,52:31,445.83,90:44:04
3,RTC,[img:flags/PL.gif],32,Bartosz Fudali,M,35,Wolsztyn,POL,103,53:25,429.16,80:26:55
4,DNC,[img:flags/JP.gif],13,Terumichi Morishita,M,43,Toyota City,JPN,101,58:40,420.83,81:50:09
5,DNC,[img:flags/AU.gif],1,Phil Gore,M,37,Darling Downs,AUS,100,58:45,416.66,77:17:47
...,...,...,...,...,...,...,...,...,...,...,...,...
71,OVER,[img:flags/PL.gif],35,Patryk Swietochowski,M,18,Minsk Mazowiecki,POL,25,55:20,104.16,21:08:19
72,OVER,[img:flags/VN.gif],71,Van Da Bui,M,45,Ho Chi Minh,VNM,24,45:47,100.00,17:48:18
73,RTC,[img:flags/BR.gif],72,Rene Romualdo Cunha,M,44,Santa Cruz Do Capibaribe,BRA,23,48:27,95.83,16:21:39
74,OVER,[img:flags/FR.gif],47,Fabrice Puaud,M,55,Mignaloux-Beauvoir,FRA,10,56:58,41.66,8:23:05


In [20]:
df=df.drop(['',"City","State"],axis=1) #won't use 

In [21]:
url2="https://my1.raceresult.com/266852/RRPublish/data/list?key=c32693b41c83792c4af8a8bcf61c4aa0&listname=Result Lists|Lap Details&page=results&contest=0&r=all&l=0"
r=requests.get(url2)
json_laps=r.json()    #getting the lap times

In [27]:
json.dumps(json_laps, indent=4); #to visualize, remove semicolon

In [28]:
for i in range(1,109): #add empty columns for specific lap times for future use
    df["Lap#"+str(i)]=pd.NA

In [29]:
df

,Place,Bib,Name,Gender,Age,Laps,Last Loop,Miles,RaceTime,Lap#1,...,Lap#99,Lap#100,Lap#101,Lap#102,Lap#103,Lap#104,Lap#105,Lap#106,Lap#107,Lap#108
1,1.,5,Harvey Lewis,M,47,108,47:31,450.00,94:16:36,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,DNC,24,Ihor Verys,M,29,107,52:31,445.83,90:44:04,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,RTC,32,Bartosz Fudali,M,35,103,53:25,429.16,80:26:55,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,DNC,13,Terumichi Morishita,M,43,101,58:40,420.83,81:50:09,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,DNC,1,Phil Gore,M,37,100,58:45,416.66,77:17:47,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,OVER,35,Patryk Swietochowski,M,18,25,55:20,104.16,21:08:19,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
72,OVER,71,Van Da Bui,M,45,24,45:47,100.00,17:48:18,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
73,RTC,72,Rene Romualdo Cunha,M,44,23,48:27,95.83,16:21:39,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
74,OVER,47,Fabrice Puaud,M,55,10,56:58,41.66,8:23:05,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [32]:
data=json_laps["data"]
pattern=re.compile(r"(\/\/\/)(.+)(\/\/\/)")   #pattern to look for after analysis of the json
for a in data:
    match = pattern.search(a)    
    name=match.group(2)
    times=[]
    for l in data[a]:
        value=l[3] #format mm:ss, conversion en secondes
        sec=int(value[:2])*60+int(value[3:])
        times.append(sec)
    n_laps=len(times)
    df.loc[df.Name==name,df.columns[9:9+n_laps]]=times

    
    

In [267]:
df

,Place,Bib,Name,Gender,Age,Laps,Last Loop,Miles,RaceTime,Lap#1,...,Lap#99,Lap#100,Lap#101,Lap#102,Lap#103,Lap#104,Lap#105,Lap#106,Lap#107,Lap#108
1,1.,5,Harvey Lewis,M,47,108,47:31,450.00,94:16:36,3074,...,3228,3264,3304,3267,3261,3245,3266,3182,3031,2851
2,DNC,24,Ihor Verys,M,29,107,52:31,445.83,90:44:04,3216,...,2900,3113,3018,3234,3029,2919,2959,3089,3151,<NA>
3,RTC,32,Bartosz Fudali,M,35,103,53:25,429.16,80:26:55,2937,...,3106,3176,3142,3235,3205,<NA>,<NA>,<NA>,<NA>,<NA>
4,DNC,13,Terumichi Morishita,M,43,101,58:40,420.83,81:50:09,2741,...,3333,3336,3520,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,DNC,1,Phil Gore,M,37,100,58:45,416.66,77:17:47,3092,...,3295,3525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,OVER,35,Patryk Swietochowski,M,18,25,55:20,104.16,21:08:19,2990,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
72,OVER,71,Van Da Bui,M,45,24,45:47,100.00,17:48:18,2462,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
73,RTC,72,Rene Romualdo Cunha,M,44,23,48:27,95.83,16:21:39,2550,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
74,OVER,47,Fabrice Puaud,M,55,10,56:58,41.66,8:23:05,2877,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [33]:
df["Age"]= df['Age'].astype(int) #Conversion des entiers
df["Bib"]= df['Bib'].astype(int)
df["Laps"]= df['Laps'].astype(int)
df["Miles"]= df['Miles'].astype(float)
for i in range(1,109):
    df['Lap#'+str(i)]=df['Lap#'+str(i)].astype('Int64') #Nullable integer



In [34]:
#Convert Last Loop in Seconds
df.loc[:,"Last Loop"]=df.loc[:,"Last Loop"].apply(lambda x: 60*int(x[:2])+int(x[3:]))

In [35]:
df["Last Loop"]= df['Last Loop'].astype(int)

In [36]:
df.to_csv('data.csv')